In [1]:
# Imports modules
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx
import folium
from pyproj import CRS
from folium.plugins import MarkerCluster
from folium.plugins import FeatureGroupSubGroup

%matplotlib inline

In [2]:
# Reads national park polygon data, Flickd post point data and continent csv file data
national_parks_fp = r"C:\Users\jooel\GISproject\data\WDPA_national_parks.shp"
flickr_fp = r'C:\Users\jooel\GISproject\data\flickr-in-national-parks.gpkg'
continents_fp = r"C:\Users\jooel\GISproject\data\continent.csv"

In [3]:
# Reads national park polygon data
national_parks = gpd.read_file(national_parks_fp, encoding='utf-8')

In [4]:
# Checks the national park data
print(len(national_parks))
national_parks.head()

2556


,WDPAID,WDPA_PID,PA_DEF,NAME,ORIG_NAME,DESIG,DESIG_ENG,DESIG_TYPE,IUCN_CAT,INT_CRIT,...,OWN_TYPE,MANG_AUTH,MANG_PLAN,VERIF,METADATAID,SUB_LOC,PARENT_ISO,ISO3,Land_area,geometry
0,20166.0,20166,1,Mbam et Djerem,Mbam et Djerem,Parc national,National Park,National,II,Not Applicable,...,State,State/WCS,Yes,State Verified,1701,CM-AD;CM-ES;CM-CE,CMR,CMR,4316.448,"POLYGON ((12.75027 6.20935, 12.75068 6.20880, ..."
1,1245.0,1245,1,Lobéké,Lobéké,Parc national,National Park,National,II,Not Applicable,...,State,State/WWF,Yes,State Verified,1701,CM-ES,CMR,CMR,2169.021,"POLYGON ((15.61263 2.49408, 15.61426 2.49408, ..."
2,607.0,607,1,Bénoué,Bénoué,Parc national,National Park,National,II,Not Applicable,...,State,State/WWF,Yes,State Verified,1701,CM-NO,CMR,CMR,1993.124,"POLYGON ((13.70366 8.66060, 13.71819 8.65609, ..."
3,606.0,606,1,Bouba Ndjida,Bouba Ndjida,Parc national,National Park,National,II,Not Applicable,...,State,State,Yes,State Verified,1701,CM-NO,CMR,CMR,2129.120,"POLYGON ((14.87231 8.46135, 14.88299 8.45150, ..."
4,1241.0,1241,1,Faro,Faro,Parc national,National Park,National,II,Not Applicable,...,State,State,Yes,State Verified,1701,CM-NO,CMR,CMR,3519.857,"POLYGON ((12.74226 8.47388, 12.73558 8.46035, ..."


In [5]:
# Saves national park data as csv
# outfp_parkdata = r"C:\Users\jooel\GISproject\data\national_parks.csv"
# national_parks.to_csv(outfp_parkdata, encoding='utf-8')

In [5]:
# Reads the Flickr post data
flickr_data = gpd.read_file(flickr_fp)

In [6]:
# Checks the Flickr data
print(len(flickr_data))
flickr_data.head()

2346541


,id,title,description,date_taken,date_posted,filename,page_url,photo_url,photo_hash,lat,...,geo_region,geo_country,user_id,user_name,user_realname,user_location,media,media_downloaded,kba_bbox,geometry
0,46131699642,Socorro Trip 2018,,2018-11-28T17:41:09,2018-12-05T01:58:38,46131699642_37f4bae166_m.jpg,,https://farm5.staticflickr.com/4861/4613169964...,None,18.992500,...,,,54091131@N00,W.Worresch,,,None,False,"-140.0000000000,-10.0000000000,-110.0000000000...",POINT (-112.07750 18.99250)
1,31243073717,Socorro Trip 2018,,2018-11-27T16:17:48,2018-12-05T01:58:53,31243073717_7edb315d02_m.jpg,,https://farm5.staticflickr.com/4812/3124307371...,None,18.724444,...,,,54091131@N00,W.Worresch,,,None,False,"-140.0000000000,-10.0000000000,-110.0000000000...",POINT (-110.91722 18.72444)
2,41560051605,20171129-032,,2017-11-29T11:14:40,2018-05-31T01:37:30,41560051605_e26a2265b1_m.jpg,,https://farm2.staticflickr.com/1743/4156005160...,None,19.322105,...,,,7198447@N03,Justin Beevor,,,None,False,"-140.0000000000,-10.0000000000,-110.0000000000...",POINT (-110.81741 19.32211)
3,18562645474,Silvertip shark (Carcharhinus albimarginatus),,2015-06-19T08:46:44,2015-06-26T23:48:10,18562645474_0644d391a2_m.jpg,,https://farm4.staticflickr.com/3928/1856264547...,None,18.999812,...,,,67374204@N00,Elias Levy,,,None,False,"-140.0000000000,-10.0000000000,-110.0000000000...",POINT (-112.06712 18.99981)
4,18562612074,Yellowfin tuna (Thunnus albacares),,2015-06-18T16:21:38,2015-06-26T23:46:35,18562612074_1b5427f789_m.jpg,,https://farm1.staticflickr.com/485/18562612074...,None,18.999812,...,,,67374204@N00,Elias Levy,,,None,False,"-140.0000000000,-10.0000000000,-110.0000000000...",POINT (-112.06712 18.99981)


In [7]:
# Removes any duplicates 
flickr_data_cleaned = flickr_data.drop_duplicates(subset=["id", "title", "description", "date_posted", "filename", "photo_url", "geometry"]) 

In [8]:
# Checks the Flickr data
print(len(flickr_data_cleaned))
flickr_data_cleaned.head()

2333550


,id,title,description,date_taken,date_posted,filename,page_url,photo_url,photo_hash,lat,...,geo_region,geo_country,user_id,user_name,user_realname,user_location,media,media_downloaded,kba_bbox,geometry
0,46131699642,Socorro Trip 2018,,2018-11-28T17:41:09,2018-12-05T01:58:38,46131699642_37f4bae166_m.jpg,,https://farm5.staticflickr.com/4861/4613169964...,None,18.992500,...,,,54091131@N00,W.Worresch,,,None,False,"-140.0000000000,-10.0000000000,-110.0000000000...",POINT (-112.07750 18.99250)
1,31243073717,Socorro Trip 2018,,2018-11-27T16:17:48,2018-12-05T01:58:53,31243073717_7edb315d02_m.jpg,,https://farm5.staticflickr.com/4812/3124307371...,None,18.724444,...,,,54091131@N00,W.Worresch,,,None,False,"-140.0000000000,-10.0000000000,-110.0000000000...",POINT (-110.91722 18.72444)
2,41560051605,20171129-032,,2017-11-29T11:14:40,2018-05-31T01:37:30,41560051605_e26a2265b1_m.jpg,,https://farm2.staticflickr.com/1743/4156005160...,None,19.322105,...,,,7198447@N03,Justin Beevor,,,None,False,"-140.0000000000,-10.0000000000,-110.0000000000...",POINT (-110.81741 19.32211)
3,18562645474,Silvertip shark (Carcharhinus albimarginatus),,2015-06-19T08:46:44,2015-06-26T23:48:10,18562645474_0644d391a2_m.jpg,,https://farm4.staticflickr.com/3928/1856264547...,None,18.999812,...,,,67374204@N00,Elias Levy,,,None,False,"-140.0000000000,-10.0000000000,-110.0000000000...",POINT (-112.06712 18.99981)
4,18562612074,Yellowfin tuna (Thunnus albacares),,2015-06-18T16:21:38,2015-06-26T23:46:35,18562612074_1b5427f789_m.jpg,,https://farm1.staticflickr.com/485/18562612074...,None,18.999812,...,,,67374204@N00,Elias Levy,,,None,False,"-140.0000000000,-10.0000000000,-110.0000000000...",POINT (-112.06712 18.99981)


In [9]:
# Reads the continent information data
continents = pd.read_csv(continents_fp)

In [10]:
# Checks the continent data
print(len(continents))
continents.head()

249


,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region_remastered,region_bigger,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,Southern Asia,Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,Europe,Europe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Asia
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,Europe,Europe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,Northern Africa,Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,Oceania,Oceania,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Africa


In [11]:
# Selects needed columns from the continent data
continents = continents[['name', 'alpha-2', 'alpha-3', 'region_remastered', 'region_bigger']]

In [12]:
# Selects needed columns from the national country data
national_parks = national_parks[['NAME','geometry', 'GIS_AREA', 'Land_area', 'PARENT_ISO', 'ISO3']]

In [13]:
# Renames continent data country code column to match national park country code column
continents = continents.rename(columns={"alpha-3" : "ISO3"})

In [14]:
# Changes the coordinate systems of national park data and flickr data to match each other
national_parks.to_crs(epsg=4326)
flickr_data_cleaned.crs = national_parks.crs

C:\Users\jooel\Anaconda3\envs\gis\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [15]:
# Checks that the coordinate systems are the same
flickr_data_cleaned.crs == national_parks.crs

True

In [16]:
# Merges national parks data with continents data
national_parks = pd.merge(national_parks, continents, on='ISO3')

In [17]:
# Checks the merged national parks data
print(len(national_parks))
national_parks.head()

2556


,NAME,geometry,GIS_AREA,Land_area,PARENT_ISO,ISO3,name,alpha-2,region_remastered,region_bigger
0,Mbam et Djerem,"POLYGON ((12.75027 6.20935, 12.75068 6.20880, ...",4316.447952,4316.448,CMR,CMR,Cameroon,CM,Sub-Saharan Africa,Africa
1,Lobéké,"POLYGON ((15.61263 2.49408, 15.61426 2.49408, ...",2169.020609,2169.021,CMR,CMR,Cameroon,CM,Sub-Saharan Africa,Africa
2,Bénoué,"POLYGON ((13.70366 8.66060, 13.71819 8.65609, ...",1993.124336,1993.124,CMR,CMR,Cameroon,CM,Sub-Saharan Africa,Africa
3,Bouba Ndjida,"POLYGON ((14.87231 8.46135, 14.88299 8.45150, ...",2129.120121,2129.120,CMR,CMR,Cameroon,CM,Sub-Saharan Africa,Africa
4,Faro,"POLYGON ((12.74226 8.47388, 12.73558 8.46035, ...",3519.857478,3519.857,CMR,CMR,Cameroon,CM,Sub-Saharan Africa,Africa


In [18]:
# Creates a new joined dataframe of flickr data and national parks
flickr_joined = gpd.sjoin(flickr_data_cleaned, national_parks, how="inner", op="within")

In [19]:
len(flickr_data_cleaned)

2333550

In [21]:
# Check the new joined data 
print(len(flickr_joined))
flickr_joined.head()

2347617


,id,title,description,date_taken,date_posted,filename,page_url,photo_url,photo_hash,lat,...,index_right,NAME,GIS_AREA,Land_area,PARENT_ISO,ISO3,name,alpha-2,region_remastered,region_bigger
0,46131699642,Socorro Trip 2018,,2018-11-28T17:41:09,2018-12-05T01:58:38,46131699642_37f4bae166_m.jpg,,https://farm5.staticflickr.com/4861/4613169964...,None,18.992500,...,2322,Revillagigedo,148835.040328,206.837,MEX,MEX,Mexico,MX,Latin America and the Caribbean,Latin America and the Caribbean
1,31243073717,Socorro Trip 2018,,2018-11-27T16:17:48,2018-12-05T01:58:53,31243073717_7edb315d02_m.jpg,,https://farm5.staticflickr.com/4812/3124307371...,None,18.724444,...,2322,Revillagigedo,148835.040328,206.837,MEX,MEX,Mexico,MX,Latin America and the Caribbean,Latin America and the Caribbean
2,41560051605,20171129-032,,2017-11-29T11:14:40,2018-05-31T01:37:30,41560051605_e26a2265b1_m.jpg,,https://farm2.staticflickr.com/1743/4156005160...,None,19.322105,...,2322,Revillagigedo,148835.040328,206.837,MEX,MEX,Mexico,MX,Latin America and the Caribbean,Latin America and the Caribbean
3,18562645474,Silvertip shark (Carcharhinus albimarginatus),,2015-06-19T08:46:44,2015-06-26T23:48:10,18562645474_0644d391a2_m.jpg,,https://farm4.staticflickr.com/3928/1856264547...,None,18.999812,...,2322,Revillagigedo,148835.040328,206.837,MEX,MEX,Mexico,MX,Latin America and the Caribbean,Latin America and the Caribbean
4,18562612074,Yellowfin tuna (Thunnus albacares),,2015-06-18T16:21:38,2015-06-26T23:46:35,18562612074_1b5427f789_m.jpg,,https://farm1.staticflickr.com/485/18562612074...,None,18.999812,...,2322,Revillagigedo,148835.040328,206.837,MEX,MEX,Mexico,MX,Latin America and the Caribbean,Latin America and the Caribbean


In [22]:
# Checks for duplicate posts
flickr_joined_d = flickr_joined[['id', 'photo_url']]
flickr_duplicates = flickr_joined_d.groupby(['id']).count().reset_index()
flickr_duplicates = flickr_duplicates[flickr_duplicates.photo_url > 1]
print(len(flickr_duplicates))
flickr_duplicates.head()

14067


,id,photo_url
241,2934809,2
242,2957309,2
261,3033955,2
282,3612589,2
289,3813681,2


There are 14 067 posts that are inside borders of two overlapping national parks

In [23]:
# Saves Flickr joined data 
outfp_joined = r"C:\Users\jooel\GISproject\data\flickr_joined.csv"
flickr_joined.to_csv(outfp_joined)

In [66]:
# Groups Flickr data by national parks
flickr_joined_b = flickr_joined[['name', 'NAME', 'GIS_AREA', 'Land_area', 'ISO3', 'region_remastered', 'region_bigger', 'id']]
flickr_joined_b['Number_of_parks'] = 1
flickr_grouped = flickr_joined_b.groupby(['name', 'NAME', 'GIS_AREA', 'Land_area', 'ISO3', 'region_remastered', 'region_bigger', 'Number_of_parks']).count().reset_index()

C:\Users\jooel\Anaconda3\envs\gis\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [67]:
# Checks the grouped data
print(len(flickr_grouped))
flickr_grouped.head()

1972


,name,NAME,GIS_AREA,Land_area,ISO3,region_remastered,region_bigger,Number_of_parks,id
0,Algeria,National park of Theniet el had Algeria,33.352098,33.352,DZA,Northern Africa,Africa,1,4
1,Algeria,Parc National d'El-Kala,810.925517,800.266,DZA,Northern Africa,Africa,1,9
2,Algeria,Parc National de Djurdjura,164.127690,164.128,DZA,Northern Africa,Africa,1,195
3,American Samoa,Not Reported,43.412188,23.501,ASM,Oceania,Oceania,1,118
4,Argentina,Aconquija,495.151519,495.152,ARG,Latin America and the Caribbean,Latin America and the Caribbean,1,1


In [68]:
# Counts parks' post densities using the whole area
flickr_grouped['post_density_total'] = flickr_grouped['id']/flickr_grouped['GIS_AREA']

In [69]:
# Counts parks' post densities using land area
flickr_grouped['post_density_land'] = flickr_grouped['id']/flickr_grouped['Land_area']

In [70]:
# Renames name column to Country and id column to represent number of posts inside park polygon and
flickr_grouped = flickr_grouped.rename(columns={"id" : "Number_of_posts", "name" : "Country"})

In [76]:
# Selects columns for a national summary
flickr_summary_national = flickr_grouped[['Country', 'ISO3', 'Number_of_parks', 'GIS_AREA', 'region_remastered', 'region_bigger', 'Land_area', 'Number_of_posts']]

In [77]:
flickr_summary_national

,Country,ISO3,Number_of_parks,GIS_AREA,region_remastered,region_bigger,Land_area,Number_of_posts
0,Algeria,DZA,1,33.352098,Northern Africa,Africa,33.352,4
1,Algeria,DZA,1,810.925517,Northern Africa,Africa,800.266,9
2,Algeria,DZA,1,164.127690,Northern Africa,Africa,164.128,195
3,American Samoa,ASM,1,43.412188,Oceania,Oceania,23.501,118
4,Argentina,ARG,1,495.151519,Latin America and the Caribbean,Latin America and the Caribbean,495.152,1
...,...,...,...,...,...,...,...,...
1967,Zimbabwe,ZWE,1,412.756870,Sub-Saharan Africa,Africa,412.757,229
1968,Zimbabwe,ZWE,1,1430.414583,Sub-Saharan Africa,Africa,1430.415,27
1969,Zimbabwe,ZWE,1,290.191551,Sub-Saharan Africa,Africa,290.192,17
1970,Zimbabwe,ZWE,1,26.624201,Sub-Saharan Africa,Africa,26.624,211


In [78]:
# Groups data by country code
flickr_summary_national = flickr_summary_national.groupby(['Country', 'ISO3', 'region_remastered', 'region_bigger']).sum().reset_index()

In [79]:
# Counts average densities using the whole area and land area seperately on a national scale
flickr_summary_national['post_density_national_total'] = flickr_summary_national['Number_of_posts']/flickr_summary_national['GIS_AREA']  
flickr_summary_national['post_density_national_land'] = flickr_summary_national['Number_of_posts']/flickr_summary_national['Land_area']  

In [80]:
# Checks the national summary data
print(len(flickr_summary_national))
flickr_summary_national.head()

130


,Country,ISO3,region_remastered,region_bigger,Number_of_parks,GIS_AREA,Land_area,Number_of_posts,post_density_national_total,post_density_national_land
0,Algeria,DZA,Northern Africa,Africa,3,1008.405305,997.746,208,0.206266,0.208470
1,American Samoa,ASM,Oceania,Oceania,1,43.412188,23.501,118,2.718131,5.021063
2,Argentina,ARG,Latin America and the Caribbean,Latin America and the Caribbean,31,28554.726116,28541.108,25028,0.876492,0.876911
3,Australia,AUS,Australia and New Zealand,Australia and New Zealand,504,279153.482471,277552.749,151426,0.542447,0.545576
4,Austria,AUT,Europe,Europe,20,2302.171226,2302.171,7238,3.143989,3.143989


In [81]:
# Calculates descriptive statistics on park level
flickr_summary_stats = flickr_grouped[['ISO3', 'post_density_total']]
flickr_summary_national_max = flickr_summary_stats.groupby(['ISO3']).max().reset_index()
flickr_summary_national_min = flickr_summary_stats.groupby(['ISO3']).min().reset_index()
flickr_summary_national_mean = flickr_summary_stats.groupby(['ISO3']).mean().reset_index()
flickr_summary_national_median = flickr_summary_stats.groupby(['ISO3']).median().reset_index()
flickr_summary_national_var = flickr_summary_stats.groupby(['ISO3']).var().reset_index()


In [82]:
flickr_summary_national['post_density_total_max'] = flickr_summary_national_max['post_density_total']
flickr_summary_national['post_density_total_min'] = flickr_summary_national_min['post_density_total']
flickr_summary_national['post_density_total_mean'] = flickr_summary_national_mean['post_density_total']
flickr_summary_national['post_density_total_median'] = flickr_summary_national_median['post_density_total']
flickr_summary_national['post_density_total_var'] = flickr_summary_national_var['post_density_total']

In [83]:
# Checks national summary with descriptive statistics
flickr_summary_national

,Country,ISO3,region_remastered,region_bigger,Number_of_parks,GIS_AREA,Land_area,Number_of_posts,post_density_national_total,post_density_national_land,post_density_total_max,post_density_total_min,post_density_total_mean,post_density_total_median,post_density_total_var
0,Algeria,DZA,Northern Africa,Africa,3,1008.405305,997.746,208,0.206266,0.208470,25.059723,0.000779,1.428575,0.061210,2.312837e+01
1,American Samoa,ASM,Oceania,Oceania,1,43.412188,23.501,118,2.718131,5.021063,2.718131,2.718131,2.718131,2.718131,NaN
2,Argentina,ARG,Latin America and the Caribbean,Latin America and the Caribbean,31,28554.726116,28541.108,25028,0.876492,0.876911,44779.208146,0.000222,126.191235,0.440850,4.135319e+06
3,Australia,AUS,Australia and New Zealand,Australia and New Zealand,504,279153.482471,277552.749,151426,0.542447,0.545576,781.209120,0.091467,46.442348,3.927140,3.010151e+04
4,Austria,AUT,Europe,Europe,20,2302.171226,2302.171,7238,3.143989,3.143989,0.180365,0.004093,0.058944,0.018488,5.775641e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,Virgin Islands (British),VGB,Latin America and the Caribbean,Latin America and the Caribbean,19,4.898585,4.110,839,171.273950,204.136253,13.998895,0.005991,2.053180,0.384862,1.231732e+01
126,Virgin Islands (U.S.),VIR,Latin America and the Caribbean,Latin America and the Caribbean,1,60.066030,39.565,9208,153.297962,232.730949,0.564402,0.564402,0.564402,0.564402,NaN
127,Western Sahara,ESH,Northern Africa,Africa,1,18979.914593,15269.927,121,0.006375,0.007924,97.409951,0.062414,6.710773,1.504134,4.855005e+02
128,Zambia,ZMB,Sub-Saharan Africa,Africa,12,45199.292781,45199.293,7202,0.159339,0.159339,52.221246,0.002449,4.407861,0.048668,2.267274e+02


In [84]:
# Creates summary on continental level
flickr_summary_continent = flickr_grouped[['region_remastered', 'Number_of_parks', 'GIS_AREA', 'Land_area', 'Number_of_posts']]
flickr_summary_continent = flickr_summary_continent.groupby(['region_remastered']).sum().reset_index()
flickr_summary_continent['post_density_continent_total'] = flickr_summary_continent['Number_of_posts']/flickr_summary_continent['GIS_AREA'] 
flickr_summary_continent['post_density_continent_land'] = flickr_summary_continent['Number_of_posts']/flickr_summary_continent['Land_area'] 

In [85]:
# Checks the continental sumamry data
print(len(flickr_summary_continent))
flickr_summary_continent.head()

13


,region_remastered,Number_of_parks,GIS_AREA,Land_area,Number_of_posts,post_density_continent_total,post_density_continent_land
0,Australia and New Zealand,517,308184.807369,306386.318,201723,0.654552,0.658394
1,Central Asia,4,30684.659483,30684.660,1258,0.040998,0.040998
2,Eastern Asia,41,24031.864995,16882.200,205477,8.550190,12.171222
3,Europe,338,198868.150697,184170.596,410160,2.062472,2.227066
4,Greenland,1,961673.218297,865940.545,689,0.000716,0.000796


In [86]:
# Calculates descriptive statistics on national level
flickr_summary_stats_national = flickr_summary_national[['region_remastered', 'post_density_national_total']]
flickr_summary_continent_max = flickr_summary_stats_national.groupby(['region_remastered']).max().reset_index()
flickr_summary_continent_min = flickr_summary_stats_national.groupby(['region_remastered']).min().reset_index()
flickr_summary_continent_mean = flickr_summary_stats_national.groupby(['region_remastered']).mean().reset_index()
flickr_summary_continent_median = flickr_summary_stats_national.groupby(['region_remastered']).median().reset_index()
flickr_summary_continent_var = flickr_summary_stats_national.groupby(['region_remastered']).var().reset_index()

In [87]:
flickr_summary_continent['post_density_national_total_max'] = flickr_summary_continent_max['post_density_national_total']
flickr_summary_continent['post_density_national__total_min'] = flickr_summary_continent_min['post_density_national_total']
flickr_summary_continent['post_density_national_total_mean'] = flickr_summary_continent_mean['post_density_national_total']
flickr_summary_continent['post_density_national_total_median'] = flickr_summary_continent_median['post_density_national_total']
flickr_summary_continent['post_density_national_total_var'] = flickr_summary_continent_var['post_density_national_total']

In [88]:
# Checks continental summary with descriptive statistics
flickr_summary_continent

,region_remastered,Number_of_parks,GIS_AREA,Land_area,Number_of_posts,post_density_continent_total,post_density_continent_land,post_density_national_total_max,post_density_national__total_min,post_density_national_total_mean,post_density_national_total_median,post_density_national_total_var
0,Australia and New Zealand,517,308184.807369,306386.318,201723,0.654552,0.658394,1.732508,0.542447,1.137478,1.137478,0.708122
1,Central Asia,4,30684.659483,30684.660,1258,0.040998,0.040998,0.130762,0.025374,0.066402,0.043069,0.003185
2,Eastern Asia,41,24031.864995,16882.200,205477,8.550190,12.171222,18.407767,2.187265,8.907644,6.127901,71.571403
3,Europe,338,198868.150697,184170.596,410160,2.062472,2.227066,152.663662,0.169853,14.539368,2.760752,1548.887385
4,Greenland,1,961673.218297,865940.545,689,0.000716,0.000796,0.000716,0.000716,0.000716,0.000716,NaN
5,Latin America and the Caribbean,355,607903.645515,427192.649,179121,0.294654,0.419298,171.273950,0.041087,14.928698,0.710500,1672.491413
6,Northern Africa,25,93182.628227,86449.709,10068,0.108046,0.116461,0.316405,0.006375,0.133493,0.119438,0.017056
7,Northern America,105,426093.874371,414353.356,1044805,2.452054,2.521531,7.669351,0.335371,4.002361,4.002361,26.893629
8,Oceania,13,11024.464713,3469.093,4025,0.365097,1.160246,21.319166,0.350501,5.174306,0.919331,82.328607
9,South-eastern Asia,195,182917.696167,178686.919,58239,0.318389,0.325928,0.987578,0.041833,0.462575,0.507478,0.116395


In [89]:
# Creates alternative continental level summary
flickr_summary_largecontinent = flickr_grouped[['region_bigger', 'Number_of_parks', 'GIS_AREA', 'Land_area', 'Number_of_posts']]
flickr_summary_largecontinent = flickr_summary_largecontinent.groupby(['region_bigger']).sum().reset_index()
flickr_summary_largecontinent['post_density_continent_total'] = flickr_summary_largecontinent['Number_of_posts']/flickr_summary_largecontinent['GIS_AREA'] 
flickr_summary_largecontinent['post_density_continent_land'] = flickr_summary_largecontinent['Number_of_posts']/flickr_summary_largecontinent['Land_area'] 

In [90]:
# Checks the alternative continental summary data
print(len(flickr_summary_largecontinent))
flickr_summary_largecontinent.head()

8


,region_bigger,Number_of_parks,GIS_AREA,Land_area,Number_of_posts,post_density_continent_total,post_density_continent_land
0,Africa,230,627083.376637,606017.453,180767,0.288266,0.298287
1,Asia,413,325333.072584,311648.597,326327,1.003055,1.047099
2,Australia and New Zealand,517,308184.807369,306386.318,201723,0.654552,0.658394
3,Europe,338,198868.150697,184170.596,410160,2.062472,2.227066
4,Greenland,1,961673.218297,865940.545,689,0.000716,0.000796


In [91]:
# Calculates descriptive statistics on national level
flickr_summary_stats_national_2 = flickr_summary_national[['region_bigger', 'post_density_national_total']]
flickr_summary_largecontinent_max = flickr_summary_stats_national_2.groupby(['region_bigger']).max().reset_index()
flickr_summary_largecontinent_min = flickr_summary_stats_national_2.groupby(['region_bigger']).min().reset_index()
flickr_summary_largecontinent_mean = flickr_summary_stats_national_2.groupby(['region_bigger']).mean().reset_index()
flickr_summary_largecontinent_median = flickr_summary_stats_national_2.groupby(['region_bigger']).median().reset_index()
flickr_summary_largecontinent_var = flickr_summary_stats_national_2.groupby(['region_bigger']).var().reset_index()

In [92]:
flickr_summary_largecontinent['post_density_national_total_max'] = flickr_summary_largecontinent_max['post_density_national_total']
flickr_summary_largecontinent['post_density_national__total_min'] = flickr_summary_largecontinent_min['post_density_national_total']
flickr_summary_largecontinent['post_density_national_total_mean'] = flickr_summary_largecontinent_mean['post_density_national_total']
flickr_summary_largecontinent['post_density_national_total_median'] = flickr_summary_largecontinent_median['post_density_national_total']
flickr_summary_largecontinent['post_density_national_total_var'] = flickr_summary_largecontinent_var['post_density_national_total']

In [93]:
# Checks alternative continental summary with descriptive statistics
flickr_summary_largecontinent.head(8)

,region_bigger,Number_of_parks,GIS_AREA,Land_area,Number_of_posts,post_density_continent_total,post_density_continent_land,post_density_national_total_max,post_density_national__total_min,post_density_national_total_mean,post_density_national_total_median,post_density_national_total_var
0,Africa,230,627083.376637,606017.453,180767,0.288266,0.298287,25.744129,0.000904,1.281606,0.110129,19.049848
1,Asia,413,325333.072584,311648.597,326327,1.003055,1.047099,147.844724,0.015094,7.887470,0.263914,946.099249
2,Australia and New Zealand,517,308184.807369,306386.318,201723,0.654552,0.658394,1.732508,0.542447,1.137478,1.137478,0.708122
3,Europe,338,198868.150697,184170.596,410160,2.062472,2.227066,152.663662,0.169853,14.539368,2.760752,1548.887385
4,Greenland,1,961673.218297,865940.545,689,0.000716,0.000796,0.000716,0.000716,0.000716,0.000716,NaN
5,Latin America and the Caribbean,355,607903.645515,427192.649,179121,0.294654,0.419298,171.273950,0.041087,14.928698,0.710500,1672.491413
6,Northern America,105,426093.874371,414353.356,1044805,2.452054,2.521531,7.669351,0.335371,4.002361,4.002361,26.893629
7,Oceania,13,11024.464713,3469.093,4025,0.365097,1.160246,21.319166,0.350501,5.174306,0.919331,82.328607


In [94]:
# Save all results as csv files
outfp1 = r"C:\Users\jooel\GISproject\results\flickr_parks_density.csv"
outfp2 = r"C:\Users\jooel\GISproject\results\flickr_national_density.csv"
outfp3 = r"C:\Users\jooel\GISproject\results\flickr_continental_density.csv"
outfp4 = r"C:\Users\jooel\GISproject\results\flickr_larger_continental_density.csv"
flickr_grouped.to_csv(outfp1, encoding='utf-8')
flickr_summary_national.to_csv(outfp2)
flickr_summary_continent.to_csv(outfp3)
flickr_summary_largecontinent.to_csv(outfp4)

# Add country layers
countries_gdf['species'] = None


for index_c, row_c in countries_gdf.iterrows():
    for index_s, row_s in species_group.iterrows():
        if row_c['NAME'] == row_s['NAME']:
            if row_c['species'] == None:
                row_c['species'] = row_s['species'] + ': ' + str(row_s['id'])
            else:
                row_c['species'] = row_c['species'] + ', ' + row_s['species'] + ': ' + str(row_s['id'])
                
    countries_gdf.at[index_c, 'species'] = row_c['species']
            

countries = folium.GeoJson(countries_gdf, name='Countries species',
                           style_function=lambda x: {'color':'red','fillColor':'red','weight':0.5},
                           tooltip=folium.features.GeoJsonTooltip(fields=['NAME', 'species'],
                                                            aliases = ['Country', 'Species'],
                                                            labels=True,
                                                            sticky=False))        
countries.add_to(inter_map)

print(countries_gdf)